**Librerías**

In [1]:
#| code-fold: false
# Import text from url
import requests
import re
# Colores
from termcolor import colored # Solo funciona dentro del notebook
from rich.console import Console # Este paquete logra reflejarlo en el HTML
console = Console(record=True)
# NPL en general
import spacy
# Tokenizacion
import nltk
from nltk.tokenize import word_tokenize
# Stop Words
from nltk.corpus import stopwords
# Steamming and Lemmatization
import nltk
from nltk.stem.snowball import SnowballStemmer

In [2]:
%run  "src\counter_to_html.py"

# Preproceso

## Ingesta del documento

Leemos el libro El Quijote.

In [3]:
# Enviar una solicitud GET a la URL
response = requests.get("https://www.gutenberg.org/cache/epub/2000/pg2000.txt")

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    quijote = response.text
    # print("Éxito al leer el texto.")
else:
    print("Error al leer el texto.")


Nos ocupamos únicamente de los primeros seis capítulos de la primera parte de El Quijote.

In [4]:
### Inicio del primer capítulo

# Palabra objetivo
palabra_start = "Primera parte del ingenioso hidalgo don Quijote de la Mancha"

# Expresión regular para capturar todo lo que va antes de un patrón de  búsqueda
patron = re.compile(rf"{palabra_start}\s*(.*)", re.DOTALL)

coincidencia = patron.search(quijote)

if coincidencia:
    quijote = coincidencia.group(1) # 0 incluye la palabra del search, 1 no lo incluye
    console.print("[bold]Inicio del primer capítulo:\n\n[/bold]", quijote[:251])
    # print("Inicio del texto:\n\n", quijote[:251])
else:
    console.print("[bold]Inicio del texto:\n\n[/bold]", quijote[:251])
    

### Inicio del sépimo capítulo (y final del sexto)
palabra_end = "Capítulo VII"

# patron = re.compile(rf"(.*)\b{re.escape(palabra_end)}\b", re.DOTALL) # .* es codicioso ("greedy"): captura la mayor cantidad posible de caracteres antes de la coincidir con la palabra, captura hasta la última aparición de la palabra buscada
patron = re.compile(rf"(.*?)\b{re.escape(palabra_end)}\b", re.DOTALL) # .* es no codicioso ("lazy"): captura la menor cantidad posible de caracteres y evitar que el resto del texto cooincida, se detiene en la primera coincidencia

coincidencia = patron.search(quijote)

if coincidencia:
    quijote = coincidencia.group(0)
    console.print("[bold]Final del sexto capítulo:\n\n[/bold]", quijote[-231:])
    quijote = coincidencia.group(1) # Elimino el patrón de búsqueda
else:
    console.print("[bold]Final del texto:\n\n[/bold]", quijote[-231:])


Inicio del primer capítulo:

 Capítulo primero. Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua

Final del sexto capítulo:

 o el cura en oyendo el nombre— si tal libro hubiera
mandado quemar; porque su autor fue uno de los famosos poetas del mundo, no
sólo de España, y fue felicísimo en la tradución de algunas fábulas de
Ovidio.




Capítulo VII

## Carga del modelo

Descarga y carga del modelo de lenguaje a través de la librería spaCy

In [5]:
# !python -m spacy download es_core_news_md

In [6]:
#| include: false
# import es_core_news_md
# nlp = es_core_news_md.load()

In [7]:
import spacy
nlp = spacy.load("es_core_news_md")

## Fragmentación de la información

Podemos dividir el texto en párrafos.

In [8]:
#| layout-ncol: 3
for parrafo in list(nlp(quijote).sents)[:3]: # No es necesario pasarlo a una lista, solo para coger un número de ejemplos
    console.print("[blue]\nSiguiente párrafo:\n\n[/blue]", parrafo)
    #print(parrafo)

Siguiente párrafo:

 Capítulo primero.

Siguiente párrafo:

 Que trata de la condición y ejercicio del famoso hidalgo
don Quijote de la Mancha

En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor.

Siguiente párrafo:

 Una olla de algo más vaca que carnero,
salpicón las más noches, duelos y quebrantos los sábados, lantejas los
viernes, algún palomino de añadidura los domingos, consumían las tres
partes de su hacienda.

Y los párrafos en palabras.

In [9]:
for parrafo in list(nlp(quijote).sents)[:1]:
    console.print("[blue]\nSiguiente párrafo:\n[/blue]")
    # print(colored("\nSiguiente párrafo:\n",'blue'))
    for palabra in parrafo:
        print(palabra)

Siguiente párrafo:

Capítulo
primero
.


# Pipeline NPL

In [10]:
text = quijote

## Tokenization

In [11]:
tokens = word_tokenize(text)
print(tokens[:10])

['Capítulo', 'primero', '.', 'Que', 'trata', 'de', 'la', 'condición', 'y', 'ejercicio']


## Stop Words

In [12]:
stop_words = set(stopwords.words('spanish'))
tokens = [w for w in tokens if w.lower() not in stop_words]
print(tokens[:10])

['Capítulo', 'primero', '.', 'trata', 'condición', 'ejercicio', 'famoso', 'hidalgo', 'don', 'Quijote']


## Stemming and Lemmatization

https://spacy.io/models/es  
https://github.com/explosion/spacy-models/releases?q=es_core_web_md&expanded=true

In [13]:
stemmer = SnowballStemmer("spanish")
tokens = [stemmer.stem(palabra) for palabra in tokens]
tokens[:10]

['capitul',
 'primer',
 '.',
 'trat',
 'condicion',
 'ejercici',
 'famos',
 'hidalg',
 'don',
 'quijot']

## POS Tagging

Part of speech.

La librería spacy ofrece variedad de etiquetas para un token. Podemos encontrarlas aquí: https://spacy.io/api/token#attributes. Por ejemplo:

In [14]:
#| code-fold: false
propiedades = ['text', 'lang_', 'is_digit', 'is_lower', 'is_upper', 'is_sent_start', 'is_sent_end',
               'like_email', 'like_url', # Espero no encontrar ninguna de estas etiquetas en El Quijote
               'sentiment', 'sent']

In [15]:
ejemplo_01 = nlp(text)[0]

for attr in propiedades:
    print("obj.%s = %r" % (attr, getattr(ejemplo_01, attr)))

obj.text = 'Capítulo'
obj.lang_ = 'es'
obj.is_digit = False
obj.is_lower = False
obj.is_upper = False
obj.is_sent_start = True
obj.is_sent_end = False
obj.like_email = False
obj.like_url = False
obj.sentiment = 0.0
obj.sent = Capítulo primero.


In [16]:
a=nlp(text).ents[0]

In [17]:
for attr in dir(a):
    if attr != 'doc' and attr != 'sent' and attr == 'label_':
        print("obj.%s = %r" % (attr, getattr(a, attr)))

obj.label_ = 'MISC'


In [18]:
#for named_entity in nlp(text).ents:
#    print(named_entity, named_entity.label_)

In [19]:
#import spacy

#nlp = spacy.load("es_core_news_md")
doc = nlp(text)[:10]
for token in doc:
    print(token.text, token.pos_, token.tag_ )

Capítulo PROPN PROPN
primero ADJ ADJ
. PUNCT PUNCT
Que SCONJ SCONJ
trata VERB VERB
de ADP ADP
la DET DET
condición NOUN NOUN
y CCONJ CCONJ
ejercicio NOUN NOUN


## NER

Named-Entity Recognition.

Realizamos esta subtarea de NPL mediante la librería spaCy.

In [20]:
#| code-fold: false
documento = nlp(text)

Todas las entidades nombradas se encuentran en la propiedad **document.ents**.

In [21]:
for named_entity in list(documento.ents)[:10]:
    print(named_entity, named_entity.label_)

Que trata de la condición MISC
hidalgo PER
la Mancha LOC
Una olla de algo más vaca MISC
El resto della MISC
Tenía en su casa una MISC
Frisaba LOC
Quieren PER
Quijada PER
Quesada LOC


In [22]:
import pandas as pd
from collections import Counter
people = []

for named_entity in documento.ents:
    if named_entity.label_ == "PER":
        people.append(named_entity.text)

people_tally = Counter(people)

df = pd.DataFrame(people_tally.most_common(), columns=['character', 'count'])
print(df)

               character  count
0                 Andrés      7
1                Nicolás      5
2        Amadís de Gaula      4
3                 Amadís      4
4              Rocinante      4
..                   ...    ...
133    Alonso de Ercilla      1
134            Juan Rufo      1
135  Cristóbal de Virués      1
136           Lloráralas      1
137               Ovidio      1

[138 rows x 2 columns]


In [23]:
# Convertir el Counter a una tabla HTML
html_table = counter_to_html_table(people_tally)

# Mostrar la tabla HTML
display(HTML(html_table))

Elemento,Conteo
hidalgo,2
Quieren,1
Quijada,1
Feliciano de Silva,1
Aristóteles,1
Belianís,1
recebía,2
Palmerín de Ingalaterra,1
Amadís de Gaula,4
Nicolás,5




**Etiquetas NER**
Las entidades nombradas son frases que contienen los nombres de personas, organizaciones, ubicaciones, expresiones de tiempo, cantidades, entre otras. Las entidades de interés se almacenan con su etiqueta correspondiente (por ejemplo, ‘PER‘ para personas, ‘LOC‘ para lugares, ‘ORG’ para organizaciones y ‘MISC’ que se emplea como un comodín para las tres anteriores) y sus posiciones en el texto (cable aclarar que el modelo que se utiliza mediante spaCy ya tiene las etiquetas predefinidas).  

In [24]:
#| code-fold: false
getattr(nlp.get_pipe('ner'), 'labels')

('LOC', 'MISC', 'ORG', 'PER')

In [25]:
from IPython.display import Markdown, display
import re

def get_ner_in_context(keyword, document, desired_ner_labels= False):
    
    if desired_ner_labels != False:
        desired_ner_labels = desired_ner_labels
    else:
         # all possible labels
        desired_ner_labels = list(nlp.get_pipe('ner').labels)  
        
    #Iterate through all the sentences in the document and pull out the text of each sentence
    for sentence in document.sents:
        #process each sentence
        sentence_doc = nlp(sentence.text)
        for named_entity in sentence_doc.ents:
            #Check to see if the keyword is in the sentence (and ignore capitalization by making both lowercase)
            if keyword.lower() in named_entity.text.lower()  and named_entity.label_ in desired_ner_labels:
                #Use the regex library to replace linebreaks and to make the keyword bolded, again ignoring capitalization
            
                sentence_text = re.sub('\n', ' ', sentence.text)
                sentence_text = re.sub(f"{named_entity.text}", f"**{named_entity.text}**", sentence_text, flags=re.IGNORECASE)

                display(Markdown('--')) # Si pones --- te sale una línea de izq a drcha
                display(Markdown(f"**{named_entity.label_}**"))
                display(Markdown(sentence_text))

In [26]:
type(doc)

spacy.tokens.span.Span

In [27]:
get_ner_in_context('Sancho', nlp(text), desired_ner_labels = False)

--

**PER**

Ella respondió con mucha humildad que se llamaba la Tolosa, y que era hija de un remendón natural de Toledo que vivía a las tendillas de **Sancho Bienaya**, y que dondequiera que ella estuviese le serviría y le tendría por señor.

In [28]:
import spacy
nlp = spacy.load("es_core_news_md")
doc = nlp("Apple está pensando en comprar una startup de España por 5.000 €.")
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
España ORG


In [29]:
doc

Apple está pensando en comprar una startup de España por 5.000 €.

In [30]:
get_ner_in_context

<function __main__.get_ner_in_context(keyword, document, desired_ner_labels=False)>